In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import re
%matplotlib inline


data = pd.DataFrame( pd.read_excel('./breast-cancer.xls') )  # Data Import from local/ Drive file

data['inv-nodes'].to_string(index=False)

################# Data sanitisation #################
total_rows = len(data)
print("Data size before sanitisation, Total: ", total_rows)
data.dropna(inplace=True) # Drops all NA cells by row and saves it back to source data


print("Removed, Rows with NA: ", total_rows - len(data))
for x in data.index:
    if re.search( r"^[0-9]{2}-[0-9]{2}$",  str(data.loc[x, "age"]) ) is None: #Using RegEx to confirm age pattern is correct, removes row that dont conform
        data.drop(x, inplace=True)
    elif re.search(r"^(ge40)$|^(premeno)$|^(lt40)$", str(data.loc[x, "menopause"]), re.IGNORECASE) is None:  #Using RegEx to confirm menopause name is only 1 of three
        data.drop(x, inplace=True)
    elif re.search(r"^[0-9]{2}-[0-9]{2}$", str(data.loc[x, "tumor-size"]) ) is None:  #Using RegEx to confirm tumor size is formated correctly
        data.drop(x, inplace=True)
    elif re.search(r"^[0-9]{1}-[0-9]{1}$", data.loc[x, "inv-nodes"]) is None:  #Using RegEx to confirm node-caps is only yes or no Upper or lower case
            print(x)
    elif re.match(r'^(left|right)$', str(data.loc[x, "node-caps"]), re.IGNORECASE) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(x, inplace=True)
    elif re.search(r"^1$|^2$|^3$", str(data.loc[x, "breast"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(x, inplace=True)
    elif re.search(r"^(left_up|left_low|central|right_up|right_low)$", str(data.loc[x, "breast-quad"]), re.IGNORECASE) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(x, inplace=True)

print("Total removal rows: " , total_rows - len(data))
################# Data sanitisation #################
################# Data sanitisation #################


data.sample(len(data))